<a href="https://colab.research.google.com/github/chizuchizu/IOAI/blob/main/Task2/chizu_003_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata

read_access_token = userdata.get('hf_read')
write_access_token = userdata.get('hf_write')

### Dependencies

In [2]:
import importlib
import torch, transformers

if '2.3.0' not in torch.__version__:
  !pip install torch==2.3.0
if transformers.__version__!='4.41.2':
  !pip install transformers==4.41.2

if importlib.util.find_spec('datasets') is None:
  !pip install datasets==2.18.0s
  !pip install evaluate==0.4.2
  !pip install accelerate -U


If you've just installed `accelerate`, execute `Runtime > Restart session and run all` in the Colab UI menu above.

In [3]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.functional import F

import torchvision
from torchvision import datasets, transforms, models

from tqdm.auto import tqdm

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, get_scheduler
from datasets import load_dataset, Dataset, DatasetDict

import evaluate

In [4]:
# ====================================================
# CFG
# ====================================================

class CFG:
    num_workers=4
    project = "IOAI_Task3"
    name = "chizu_008_512"

    # model
    base_model_name = "google-bert/bert-base-multilingual-uncased"
    num_classes = 5

    # training
    epochs = 20

    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']

    lr = 1e-05

    # dataset
    max_length = 512

    # T4: 32
    # L4: 64
    train_batch_size = 32
    eval_batch_size = 32

    seed=42
    train=True

In [5]:
classification_dataset = load_dataset('InternationalOlympiadAI/NLP_problem', token=read_access_token)
tokenizer = AutoTokenizer.from_pretrained(CFG.base_model_name)


def transform(example_batch):
    inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
    inputs["labels"] = example_batch["label"]
    return inputs

tokenized_data = classification_dataset.with_transform(transform)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/218 [00:00<?, ? examples/s]

In [6]:
f1 = evaluate.load("f1")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(
    CFG.base_model_name, num_labels=CFG.num_classes
).cuda()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def to_device(batch, device):
    output = {}
    for k, v in batch.items():
        try:
            output[k] = v.to(device)
        except:
            output[k] = v
    return output

In [9]:


# dataset
train_dataset = tokenized_data["train"]
eval_dataset = tokenized_data["dev"]
train_loader = DataLoader(
    train_dataset,
    batch_size=CFG.train_batch_size,
    num_workers=0,
    pin_memory=True,
    shuffle=True,
    drop_last=True,
)
eval_loader = DataLoader(
    eval_dataset,
    batch_size=CFG.eval_batch_size,
    num_workers=0,
    pin_memory=True,
    shuffle=False,
)

epochs = 20
num_training_steps = epochs * len(train_loader)
optimizer = optim.AdamW(model.parameters(), lr=CFG.lr)
scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def train_one_epoch(model, scheduler, train_loader, optimizer):
    model.train()
    running_loss = 0.0
    for batch in tqdm(train_loader):
        batch = to_device(batch, device)
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            token_type_ids=batch["token_type_ids"],
            labels=batch["labels"],
        )
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

def evaluate_model(model, test_loader):
    model.eval()
    for batch in eval_loader:
        batch = to_device(batch, device)
        with torch.no_grad():
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch["token_type_ids"],
                labels=batch["labels"],
            )

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
    return f1.compute(predictions=predictions, references=batch["labels"], average='macro')

# Train and evaluate the model
model.to(device)
for i in range(CFG.epochs):
    train_one_epoch(model, scheduler, train_loader, optimizer)
    accuracy = evaluate_model(model, eval_loader)
    print(f'Epoch {i+1} {accuracy}')

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1 {'f1': 0.16}


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 2 {'f1': 0.11587301587301588}


  0%|          | 0/6 [00:00<?, ?it/s]

...

Epoch 18 {'f1': 0.42021978021978024}


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 19 {'f1': 0.40244200244200246}


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 20 {'f1': 0.40244200244200246}


# Data

In [14]:
# run the trained model on a dev/test split
data_split = "dev"
eval_out = trainer.predict(tokenized_data[data_split])
predictions = eval_out.predictions.argmax(1)
labels = eval_out.label_ids
dev_f1 = f1.compute(predictions=predictions, references=labels, average='macro')

NameError: name 'trainer' is not defined

In [ ]:
# 精度をみる

print(dev_f1)
print()
print(predictions)
print(labels)

total = 0
correct = 0
for i in range(len(predictions)):
  if predictions[i] == labels[i]:
    correct += 1
  total += 1
print()
print(correct/total)

In [ ]:
# write the predictions to a file
with open('{}_predictions.txt'.format(data_split), 'w') as outfile:
  outfile.write('\n'.join([str(p) for p in predictions.tolist()]))

In [27]:
import gc
import torch

def flush():
  gc.collect()
  torch.cuda.empty_cache()
flush()